# Предобработка и классификация текстовых данных.

# 1.Цель лабораторной работы

изучение методов предобработки и классификации текстовых данных.

# 2.Задание

1.Для произвольного предложения или текста решите следующие задачи:

Токенизация.

Частеречная разметка.

Лемматизация.

Выделение (распознавание) именованных сущностей.

Разбор предложения.

2.Для произвольного набора данных, предназначенного для классификации текстов, решите задачу классификации текста двумя способами:

Способ 1. На основе CountVectorizer или TfidfVectorizer.

Способ 2. На основе моделей word2vec или Glove или fastText.

Сравните качество полученных моделей.

Для поиска наборов данных в поисковой системе можно использовать ключевые слова "datasets for text classification".

# Обработка текста

# Токенизация

In [6]:
!pip install razdel

In [7]:
from razdel import tokenize, sentenize

In [8]:
text = 'обеспечить возможность записи информации о клиенте (номер банковской карты, имя, номер телефона и другая основная информация), поиска, контроля доступа и мониторинга, а также управления журналом регистрации'

In [9]:
n_tok_text = list(tokenize(text))
n_tok_text

[Substring(0, 10, 'обеспечить'),
 Substring(11, 22, 'возможность'),
 Substring(23, 29, 'записи'),
 Substring(30, 40, 'информации'),
 Substring(41, 42, 'о'),
 Substring(43, 50, 'клиенте'),
 Substring(51, 52, '('),
 Substring(52, 57, 'номер'),
 Substring(58, 68, 'банковской'),
 Substring(69, 74, 'карты'),
 Substring(74, 75, ','),
 Substring(76, 79, 'имя'),
 Substring(79, 80, ','),
 Substring(81, 86, 'номер'),
 Substring(87, 95, 'телефона'),
 Substring(96, 97, 'и'),
 Substring(98, 104, 'другая'),
 Substring(105, 113, 'основная'),
 Substring(114, 124, 'информация'),
 Substring(124, 125, ')'),
 Substring(125, 126, ','),
 Substring(127, 133, 'поиска'),
 Substring(133, 134, ','),
 Substring(135, 143, 'контроля'),
 Substring(144, 151, 'доступа'),
 Substring(152, 153, 'и'),
 Substring(154, 165, 'мониторинга'),
 Substring(165, 166, ','),
 Substring(167, 168, 'а'),
 Substring(169, 174, 'также'),
 Substring(175, 185, 'управления'),
 Substring(186, 194, 'журналом'),
 Substring(195, 206, 'регистраци

In [10]:
[_.text for _ in n_tok_text]

['обеспечить',
 'возможность',
 'записи',
 'информации',
 'о',
 'клиенте',
 '(',
 'номер',
 'банковской',
 'карты',
 ',',
 'имя',
 ',',
 'номер',
 'телефона',
 'и',
 'другая',
 'основная',
 'информация',
 ')',
 ',',
 'поиска',
 ',',
 'контроля',
 'доступа',
 'и',
 'мониторинга',
 ',',
 'а',
 'также',
 'управления',
 'журналом',
 'регистрации']

In [11]:
n_sen_text = list(sentenize(text))
n_sen_text

[Substring(0,
           206,
           'обеспечить возможность записи информации о клиенте (номер банковской карты, имя, номер телефона и другая основная информация), поиска, контроля доступа и мониторинга, а также управления журналом регистрации')]

In [12]:
[_.text for _ in n_sen_text], len([_.text for _ in n_sen_text])

(['обеспечить возможность записи информации о клиенте (номер банковской карты, имя, номер телефона и другая основная информация), поиска, контроля доступа и мониторинга, а также управления журналом регистрации'],
 1)

In [13]:
# Этот вариант токенизации нужен для последующей обработки
def n_sentenize(text):
    n_sen_chunk = []
    for sent in sentenize(text):
        tokens = [_.text for _ in tokenize(sent.text)]
        n_sen_chunk.append(tokens)
    return n_sen_chunk

In [14]:
n_sen_chunk = n_sentenize(text)
n_sen_chunk

[['обеспечить',
  'возможность',
  'записи',
  'информации',
  'о',
  'клиенте',
  '(',
  'номер',
  'банковской',
  'карты',
  ',',
  'имя',
  ',',
  'номер',
  'телефона',
  'и',
  'другая',
  'основная',
  'информация',
  ')',
  ',',
  'поиска',
  ',',
  'контроля',
  'доступа',
  'и',
  'мониторинга',
  ',',
  'а',
  'также',
  'управления',
  'журналом',
  'регистрации']]

# 3.1 Для произвольного набора данных, предназначенного для классификации текстов, решите задачу классификации текста двумя способами

Способ 1. На основе CountVectorizer

In [49]:
import re
import pandas as pd
import numpy as np
from typing import Dict, Tuple
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\31139\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [53]:
df=pd.read_csv("youtube.csv")
df.head()

,link,title,description,category
0,JLZlCZ0,Ep 1| Travelling through North East India | Of...,Tanya Khanijow\n671K subscribers\nSUBSCRIBE\nT...,travel
1,i9E_Blai8vk,Welcome to Bali | Travel Vlog | Priscilla Lee,Priscilla Lee\n45.6K subscribers\nSUBSCRIBE\n*...,travel
2,r284c-q8oY,My Solo Trip to ALASKA | Cruising From Vancouv...,Allison Anderson\n588K subscribers\nSUBSCRIBE\...,travel
3,Qmi-Xwq-ME,Traveling to the Happiest Country in the World!!,Yes Theory\n6.65M subscribers\nSUBSCRIBE\n*BLA...,travel
4,_lcOX55Ef70,Solo in Paro Bhutan | Tiger's Nest visit | Bhu...,Tanya Khanijow\n671K subscribers\nSUBSCRIBE\nH...,travel


In [56]:
#Только держать колонки "verified_reviews" и "feedback".
df_new = pd.DataFrame(df,columns=['description','category'])
df_new.columns = ['text','value']
df_new.head()

,text,value
0,Tanya Khanijow\n671K subscribers\nSUBSCRIBE\nT...,travel
1,Priscilla Lee\n45.6K subscribers\nSUBSCRIBE\n*...,travel
2,Allison Anderson\n588K subscribers\nSUBSCRIBE\...,travel
3,Yes Theory\n6.65M subscribers\nSUBSCRIBE\n*BLA...,travel
4,Tanya Khanijow\n671K subscribers\nSUBSCRIBE\nH...,travel


In [57]:
df_new.shape

(3599, 2)

In [58]:
#Сформируем общий словарь
vocab_list = df_new['text'].tolist()
vocab_list[1:10]

["Priscilla Lee\n45.6K subscribers\nSUBSCRIBE\n*DISCLAIMER* Please do not ride elephants when visiting any country. At the time I didn't know (yes, I was dumb) so it is shown in the video, but I do not support the elephant riding business anymore. If I could take it back I would, but instead I want to pass on the knowledge to anyone who isn't aware. Here's some info: \nSHOW MORE",
 'Allison Anderson\n588K subscribers\nSUBSCRIBE\nI spent 11 days cruising up the coast of Alaska and it was MAGICAL.\n•ALASKA BLOG POST: https://allisonanderson.com/blog/crui...\n•Adventures on INSTAGRAM http://www.instagram.com/photoallison\nSHOW MORE',
 "Yes Theory\n6.65M subscribers\nSUBSCRIBE\n*BLACK FRIDAY DROP Out Now*: http://seek-discomfort.com/yes-theory \nThis week only, with every purchase about $35, you'll get 2 free Seek Discomfort flags!\n\nCheck out our friends from Beautiful Destinations!! Their videos are INCREDIBLE:\nSHOW MORE",
 'Tanya Khanijow\n671K subscribers\nSUBSCRIBE\nHere’s presentin

In [59]:
vocabVect = CountVectorizer(
    stop_words='english',
    ngram_range=(1, 1),  #ngram_range=(1, 1) is the default
    dtype='double'
)
vocabVect.fit(vocab_list)
corpusVocab = vocabVect.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 22038


In [60]:
for i in list(corpusVocab)[1:10]:
    print('{}={}'.format(i, corpusVocab[i]))

khanijow=10716
671k=1257
subscribers=18036
subscribe=18030
journey=10329
arunachal=2552
north=13431
east=6607
india=9566


In [61]:
test_features = vocabVect.transform(vocab_list)

In [62]:
test_features

<3599x22038 sparse matrix of type '<class 'numpy.float64'>'
	with 114891 stored elements in Compressed Sparse Row format>

In [63]:
test_features.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [64]:
# Размер нулевой строки
len(test_features.todense()[0].getA1())

22038

In [65]:
# Непустые значения нулевой строки
[i for i in test_features.todense()[0].getA1() if i>0]

[1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [66]:
vocabVect.get_feature_names()[100:120]

D:\software\anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['11204',
 '11261',
 '112m',
 '113',
 '113k',
 '114',
 '114k',
 '115',
 '115k',
 '116',
 '1160',
 '116k',
 '117',
 '117k',
 '118',
 '1185',
 '119',
 '11k',
 '11m',
 '11th']

Разделим выборку на обучающую и тестовую

In [68]:
X_train, X_test, y_train, y_test = train_test_split(df_new['text'], df_new['value'], test_size=0.3, random_state=1)

In [69]:
def sentiment(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)

Используем классификатор "LogisticRegression"

In [70]:
sentiment(CountVectorizer(), LogisticRegression(C=3.0))

Метка 	 Accuracy
art_music 	 0.9253731343283582
food 	 0.8598484848484849
history 	 0.8531073446327684
travel 	 0.9191374663072777


D:\software\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Способ 2. На основе моделей word2vec

In [71]:
import gensim
from gensim.models import word2vec

In [73]:
# Подготовим корпус
corpus = []
stop_words = stopwords.words('english')
tok = WordPunctTokenizer()
for line in df_new['text'].values:
    line1 = line.strip().lower()
    line1 = re.sub("[^a-zA-Z]"," ", line1)
    text_tok = tok.tokenize(line1)
    text_tok1 = [w for w in text_tok if not w in stop_words]
    corpus.append(text_tok1)

In [74]:
corpus[:5]

[['tanya',
  'khanijow',
  'k',
  'subscribers',
  'subscribe',
  'journey',
  'arunachal',
  'north',
  'east',
  'india',
  'begins',
  'train',
  'journey',
  'guwahati',
  'murkongselek',
  'head',
  'pasighat',
  'travel',
  'companions',
  'getting',
  'started',
  'exploring',
  'tiny',
  'glimpse',
  'arunachal',
  'far',
  'markets',
  'bridges',
  'adventure',
  'get',
  'better',
  'next',
  'video',
  'show'],
 ['priscilla',
  'lee',
  'k',
  'subscribers',
  'subscribe',
  'disclaimer',
  'please',
  'ride',
  'elephants',
  'visiting',
  'country',
  'time',
  'know',
  'yes',
  'dumb',
  'shown',
  'video',
  'support',
  'elephant',
  'riding',
  'business',
  'anymore',
  'could',
  'take',
  'back',
  'would',
  'instead',
  'want',
  'pass',
  'knowledge',
  'anyone',
  'aware',
  'info',
  'show'],
 ['allison',
  'anderson',
  'k',
  'subscribers',
  'subscribe',
  'spent',
  'days',
  'cruising',
  'coast',
  'alaska',
  'magical',
  'alaska',
  'blog',
  'post',
 

In [36]:
corpus[:5]

[['tanya',
  'khanijow',
  'k',
  'subscribers',
  'subscribe',
  'journey',
  'arunachal',
  'north',
  'east',
  'india',
  'begins',
  'train',
  'journey',
  'guwahati',
  'murkongselek',
  'head',
  'pasighat',
  'travel',
  'companions',
  'getting',
  'started',
  'exploring',
  'tiny',
  'glimpse',
  'arunachal',
  'far',
  'markets',
  'bridges',
  'adventure',
  'get',
  'better',
  'next',
  'video',
  'show'],
 ['priscilla',
  'lee',
  'k',
  'subscribers',
  'subscribe',
  'disclaimer',
  'please',
  'ride',
  'elephants',
  'visiting',
  'country',
  'time',
  'know',
  'yes',
  'dumb',
  'shown',
  'video',
  'support',
  'elephant',
  'riding',
  'business',
  'anymore',
  'could',
  'take',
  'back',
  'would',
  'instead',
  'want',
  'pass',
  'knowledge',
  'anyone',
  'aware',
  'info',
  'show'],
 ['allison',
  'anderson',
  'k',
  'subscribers',
  'subscribe',
  'spent',
  'days',
  'cruising',
  'coast',
  'alaska',
  'magical',
  'alaska',
  'blog',
  'post',
 

In [76]:
assert df_new.shape[0]==len(corpus)

In [77]:
%time model_imdb = word2vec.Word2Vec(corpus, workers=4, min_count=10, window=10, sample=1e-3)

Wall time: 304 ms


In [79]:
# Проверим, что модель обучилась
print(model_imdb.wv.most_similar(positive=['adventure'], topn=10))

[('huge', 0.9954676628112793), ('rest', 0.9946334958076477), ('big', 0.9945908784866333), ('order', 0.9945046901702881), ('peace', 0.9945025444030762), ('quick', 0.9943442344665527), ('headed', 0.9942252039909363), ('eleven', 0.9940177202224731), ('dream', 0.9939081072807312), ('chen', 0.9938148856163025)]


In [80]:
class EmbeddingVectorizer(object):
    '''
    Для текста усредним вектора входящих в него слов
    '''
    def __init__(self, model):
        self.model = model
        self.size = model.vector_size

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([np.mean(
            [self.model[w] for w in words if w in self.model] 
            or [np.zeros(self.size)], axis=0)
            for words in X])

In [82]:
sentiment(EmbeddingVectorizer(model_imdb.wv), LogisticRegression(C=3.0))

D:\software\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Метка 	 Accuracy
art_music 	 0.4925373134328358
food 	 0.2803030303030303
history 	 0.13559322033898305
travel 	 0.5876010781671159


In [83]:
sentiment(EmbeddingVectorizer(model_imdb.wv), LogisticRegression(C=5.0))

D:\software\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Метка 	 Accuracy
art_music 	 0.48880597014925375
food 	 0.3712121212121212
history 	 0.1638418079096045
travel 	 0.522911051212938


In [85]:
from sklearn.neighbors import KNeighborsClassifier
sentiment(EmbeddingVectorizer(model_imdb.wv), KNeighborsClassifier(n_neighbors=5))

Метка 	 Accuracy
art_music 	 0.5522388059701493
food 	 0.4090909090909091
history 	 0.3446327683615819
travel 	 0.40431266846361186


In [43]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [47]:
# Обучающая и тестовая выборки
boundary = 3000
X_train = corpus[:boundary] 
X_test = corpus[boundary:]
y_train = imdb_df.category.values[:boundary]
y_test = imdb_df.category.values[boundary:]

In [48]:
sentiment(EmbeddingVectorizer(model_imdb.wv), LogisticRegression(C=5.0))

Метка 	 Accuracy
art_music 	 0.6666666666666666
history 	 0.0


D:\software\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [87]:
df_new[df_new['value']=='history']

,text,value
3006,Ranveer Allahbadia\n2.53M subscribers\nSUBSCRI...,history
3007,Knowledgia\n650K subscribers\nSUBSCRIBE\nTHE H...,history
3008,Let's Crack UPSC CSE\n4.71M subscribers\nSUBSC...,history
3009,Abhijit Chavda\n74.8K subscribers\nSUBSCRIBE\n...,history
3010,PDF visuals\n98.8K subscribers\nSUBSCRIBE\nHer...,history
...,...,...
3594,CrashCourse\n12.4M subscribers\nSUBSCRIBE\nThe...,history
3595,Publications Office of the European Union\n3.2...,history
3596,History Time\n619K subscribers\nSUBSCRIBE\n- W...,history
3597,Mr. Raymond's Civics and Social Studies Academ...,history
